Compare with Tide Gauges
========================

This ({nb-download}`notebook <Compare-Tide-Gauges.ipynb>`) compares model predictions with tide gauge data provided by the [University of Hawaii Sea Level Center](https://uhslc.soest.hawaii.edu/network/)

OTIS format tidal solutions provided by Oregon State University and ESR  
- [http://volkov.oce.orst.edu/tides/region.html](http://volkov.oce.orst.edu/tides/region.html) 
- [https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/](https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/)
- [ftp://ftp.esr.org/pub/datasets/tmd/](ftp://ftp.esr.org/pub/datasets/tmd/)  

Global Tide Model (GOT) solutions provided by Richard Ray at GSFC  
- [https://earth.gsfc.nasa.gov/geo/data/ocean-tide-models](https://earth.gsfc.nasa.gov/geo/data/ocean-tide-models)

Finite Element Solution (FES) provided by AVISO  
- [https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html](https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html)
        
## Python Dependencies
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/)  
 - [matplotlib: Python 2D plotting library](http://matplotlib.org/)  

## Program Dependencies

- `crs.py`: Coordinate Reference System (CRS) routines  
- `io.model.py`: retrieves tide model parameters for named tide models  
- `io.OTIS.py`: extract tidal harmonic constants from OTIS tide models  
- `io.ATLAS.py`: extract tidal harmonic constants from ATLAS netcdf models  
- `io.GOT.py`: extract tidal harmonic constants from GOT tide models  
- `io.FES.py`: extract tidal harmonic constants from FES tide models  

This notebook uses Jupyter widgets to set parameters for calculating the tidal values.  

## Load modules

In [ ]:
from __future__ import print_function

import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import IPython.display
import ipywidgets

# import tide programs
import pyTMD.tools
import pyTMD.compute
import pyTMD.utilities
import timescale
# autoreload
%load_ext autoreload
%autoreload 2

## Set parameters for program

- Model directory  
- Tide model  

In [ ]:
# available model list
model_list = sorted(pyTMD.io.model.ocean_elevation())
# display widgets for setting directory and model
TMDwidgets = pyTMD.tools.widgets()
TMDwidgets.model.options = model_list
TMDwidgets.model.value = 'GOT4.10'
TMDwidgets.VBox([
    TMDwidgets.directory,
    TMDwidgets.model,
    TMDwidgets.compress,
])

## Find Tide Gauge Data

In [ ]:
# remote directory with tide gauge data
HOST = 'https://uhslc.soest.hawaii.edu/data/netcdf/rqds/global/hourly/'
f = pyTMD.utilities.uhslc_list(HOST, pattern=r'(.*?).nc', sort=True)
# create dropdown with all tide gauge data
TMDwidgets.gauges = ipywidgets.Dropdown(
    options=f,
    value=f[0],
    description='Tide Gauges:',
    disabled=False,
    style=TMDwidgets.style,
)
display(TMDwidgets.gauges)

## Download and Read Tide Gauge Data

In [18]:
# open tide gauge data
fid = pyTMD.utilities.from_http([HOST,TMDwidgets.gauges.value])
with netCDF4.Dataset(fid.filename, memory=fid.read()) as fileID:
    # read time and station name
    delta_time = fileID.variables['time'][:].squeeze()
    date_string = fileID.variables['time'].units
    station_name = netCDF4.chartostring(fileID.variables['station_name'][:])
    # get latitude and longitude
    lat = fileID.variables['lat'][:]
    lon = fileID.variables['lon'][:]
    # get sea level heights
    sea_level = fileID.variables['sea_level'][:].squeeze()
# convert time
epoch, to_sec = timescale.time.parse_date_string(date_string)
ts = timescale.from_deltatime(delta_time*to_sec, epoch=epoch)
deltat = ts.tt_ut1
# remove the mean and convert to meters
h = (sea_level - sea_level.mean())/1000.0

## Predict tides at measurement times

In [19]:
# calculate tide elevations
tide = pyTMD.compute.tide_elevations(lon, lat, ts.to_datetime(),
    DIRECTORY=TMDwidgets.directory.value, 
    MODEL=TMDwidgets.model.value, TYPE='time series', 
    EPSG=4326, TIME='datetime', EXTRAPOLATE=True).squeeze()

## Compare predictions and measured values

In [ ]:
%matplotlib widget
fig, ax = plt.subplots()
ax.plot(ts.year, h, label='Tide Gauge Data', lw=2)
ax.plot(ts.year, tide, label='Tide Prediction')
title = f'{station_name[0]} ({lat.data[0]:0.2f}\u00B0N, {lon.data[0]:0.2f}\u00B0E)'
ax.set_title(title)
lgd = ax.legend(frameon=False)
lgd.get_frame().set_alpha(1.0)
for line in lgd.get_lines():
    line.set_linewidth(6)
plt.tight_layout()
plt.show()